In [82]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

import cv2
from imutils import build_montages



In [83]:
# grab the Fashion MNIST dataset (if this is your first time running
# this the dataset will be automatically downloaded)
print("[INFO] loading Fashion MNIST...")
((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()

# use "channels last" ordering, so the design
# matrix shape should be: num_samples x rows x columns x depth
# channels last is the default format in tf.  
# it can be changed by editing ~/.keras/keras.json or
# specifying the the data_format in the conv2D layers
trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
testX = testX.reshape((testX.shape[0], 28, 28, 1))

# scale data to the range of [0, 1]
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

# one-hot encode the training and testing labels
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

# initialize the label names
labelNames = ["top", "trouser", "pullover", "dress", "coat",
"sandal", "shirt", "sneaker", "bag", "ankle boot"]


[INFO] loading Fashion MNIST...


In [84]:
import os

model_path = 'data/fmnist_cnn.hdf5'

assert os.path.isfile(model_path), print(f'File not found at: {model_path}')
    
model = load_model(model_path)



In [85]:
## show some test images

# initialize our list of output images
images = []

# randomly select a few testing fashion items
for i in np.random.choice(np.arange(0, len(testY)), size=(16,)):
    # classify the clothing
    probs = model.predict(testX[np.newaxis, i])
    prediction = probs.argmax(axis=1)
    label = labelNames[prediction[0]]
    
    image = (testX[i] * 255).astype("uint8")

    # initialize the text label color as green (correct)
    color = (0, 255, 0)

    # otherwise, the class label prediction is incorrect
    if prediction[0] != np.argmax(testY[i]):
        color = (0, 0, 255)
    
    # merge the channels into one image and resize the image from
    # 28x28 to 96x96 so we can better see it and then draw the
    # predicted label on the image
    image = cv2.merge([image] * 3)
    image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
    cv2.putText(image, label, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75,
        color, 2)

    # add the image to our list of output images
    images.append(image)

# construct the montage for the images
montage = build_montages(images, (96, 96), (4, 4))[0]
written = cv2.imwrite('img/fashion_mnist.png', montage)



![montage](img/fashion_mnist.png)

In [93]:
img_number = np.random.randint(len(testX))
class_probabilities = model.predict(np.expand_dims(testX[img_number], 0))

N_classes = 10
class_probabilities = class_probabilities.reshape(N_classes)
class_decision = np.argmax(class_probabilities)


In [98]:
for m in range(N_classes):
    if m == class_decision: 
        print(f'Probability of class[{m}]:  {class_probabilities[m] : 5.3e} :: <class decision>')
    else:
        print(f'Probability of class[{m}]:  {class_probabilities[m] : 5.3e}')

correct_class = np.argmax(testY[img_number])
print(f'\nCorrect class: {correct_class}\n')
if correct_class != class_decision:
    print('INCORRECT classification')
else:
    print('CORRECT classification')

Probability of class[0]:   1.227e-02
Probability of class[1]:   5.907e-06
Probability of class[2]:   1.289e-04
Probability of class[3]:   9.832e-01 :: <class decision>
Probability of class[4]:   4.875e-04
Probability of class[5]:   1.085e-06
Probability of class[6]:   3.873e-03
Probability of class[7]:   8.644e-06
Probability of class[8]:   3.540e-06
Probability of class[9]:   3.105e-06

Correct class: 3

CORRECT classification
